In [3]:
from os import path
import glob
import os
import struct
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np
import requests
from matplotlib import image

cat = 'bottle'
batch_name = 'batch-1'
num = 30
video_id = cat+'/'+batch_name+'/'+str(num)
v_id = cat+'_'+batch_name+'_'+str(num)

print(video_id, v_id)

download_dir = './Objectron_data/'+v_id
public_url = "https://storage.googleapis.com/objectron"
os.makedirs(download_dir, exist_ok=True)
video_filename = f'{public_url}/videos/{str(video_id)}/video.MOV'
public_url = "https://storage.googleapis.com/objectron"
urls = [f'{public_url}/videos/{str(video_id)}/video.MOV']
for url in urls:
    with open(path.join(download_dir, path.basename(url)), "wb") as f:
        f.write(requests.get(url).content)
        
url_sfm_ =  f'{public_url}/videos/{str(video_id)}/sfm_arframe.pbdata'
url_annotation = f'{public_url}/annotations/{str(video_id)}.pbdata'

with open(path.join(download_dir, v_id+'.pbdata'), "wb") as f:
    f.write(requests.get(url_annotation).content)

with open(path.join(download_dir, v_id+'_sfm_arframe.pbdata'), "wb") as f:
    f.write(requests.get(url_sfm_).content)


bottle/batch-1/30 bottle_batch-1_30


In [4]:
folder = '/home/zubair/nerf_pl/Objectron_data/'+str(v_id)
os.listdir(folder)

['video.MOV',
 'bottle_batch-1_30.pbdata',
 'sfm_arframe.pbdata',
 'bottle_batch-1_30_sfm_arframe.pbdata',
 '30.pbdata']

In [ ]:
# cd ./Objectron_data/laptop_batch-1_4
os.makedirs('/home/zubair/nerf_pl/Objectron_data/'+str(v_id)+'/images_12')

video_filename = '/home/zubair/nerf_pl/Objectron_data/'+str(v_id)+'/video.MOV'
out_filename = '/home/zubair/nerf_pl/Objectron_data/'+str(v_id)+'/images_12/%05d.png'
print(video_filename,out_filename)
os.system("ffmpeg -i {0} -vsync vfr -vf scale=iw/12:ih/12 -start_number 0 {1}".format(video_filename, out_filename))
# mkdir -p "images_12"
# ffmpeg -i file -vsync vfr -vf "scale=iw/12:ih/12" -start_number 0 "images_12/%05d.png"

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
im_filename = '/home/zubair/nerf_pl/Objectron_data/'+str(v_id)+'/images_12/'+'00000.png'
im = cv2.imread(im_filename)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)
# class_names = MetadataCatalog.get("mydataset").thing_classes
# pred_class_names = list(map(lambda x: class_names[x], outputs["instances"].pred_classes))
# print(pred_class_names)

In [ ]:
import matplotlib.pyplot as plt
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(out.get_image()[:, :, ::-1])
plt.show()
# cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
from detectron2.data import MetadataCatalog
for i, name in enumerate(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes):
    print(i, name)

In [ ]:
class_name_to_id = {'laptop': 63, 'bottle': 39, 'chair': 56}
class_name = 'chair'
class_id = class_name_to_id[class_name]

def get_valid_masks_index(mask_out, classes, class_id):
    areas = []
    new_masks = []
    for m in range(mask_out.shape[0]):
        if classes[m] == class_id:
            pos = np.where(mask_out[m, :, :]>0)
            center_x = np.average(pos[0])
            center_y = np.average(pos[1])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            areas.append(((ymax-ymin)* (xmax-xmin))/100)
            new_masks.append(mask_out[m])
    index = np.argmax(areas)
    return new_masks[index]

def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image


def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    # random.shuffle(colors)
    return colors

In [ ]:
import colorsys
mask_out = outputs["instances"].pred_masks.cpu().numpy()
classes = outputs["instances"].pred_classes.cpu().numpy()
mask_out = get_valid_masks_index(mask_out, classes, class_id)
print(mask_out.shape)
seg_masks = mask_out.astype(np.uint8)
colors = random_colors(mask_out.shape[0])
im = apply_mask(im, seg_masks, colors[0])
    
plt.imshow(mask_out)
plt.show()
plt.imshow(im)
plt.show()

In [ ]:
print('mask_00286.png'.split('_'))